In [1]:
import sys
import win32com.client as win32
import time
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import uic
import os
import numpy as np
import math
from PyQt5.QtGui import *
from PIL import Image
from pdf2image import convert_from_path
import errno
import re

In [2]:
#------------------------------------------------ 문장 추출 함수 ---------------------------------------------------
def title_check(sentence):
    hanguls = ['가','나','다','라','마','바','사','아','자','차','카','파','타','하']
    if sentence[0:3] == '7. ': return 1
    if sentence[0:2] == '7.' and sentence[2:sentence.find('.',2)].isnumeric() and sentence[sentence.find('.',2):sentence.find('.',2)+2] == '. ': return 2
    if (sentence[0:1] in hanguls) and sentence[1:3] == '. ': return 3
    
    print(sentence[0:sentence.find(')')])
    print(re.sub(' ','',sentence[0:sentence.find(')')]))
    print(re.sub(r'[0-9]','',re.sub(' ','',sentence[0:sentence.find(')')])))
    if sentence.find(')') > 0 and re.sub(r'[0-9]','',re.sub(' ','',sentence[0:sentence.find(')')])) == '' : return 4
    return 0
    

In [3]:
def Sentence_Extract(txt_path, option=0):

    # 해당 경로의 텍스트 파일 데이터 읽기
    try:
        with open(txt_path, 'r', encoding = 'cp949') as f:
            data = f.read()
    except: 
        with open(txt_path, 'r', encoding = 'utf8') as f:
            data = f.read()

    # 데이터 전처리(양식번호, 성적서 번호 등 삭제)
    
    for i in range(10):
        data = data.replace('\n\n','\n')
    for i in range(10):
        data = data.replace('\n'+' '*i+'\n','\n')  

    data = data.split('\n')
    
    sentences = []
    option_data = []
    bcolors = []
    root_report_num = ''
    flag = False
    for index, sentence in enumerate(data):
        
        bcolors.append((255,255,255))
        if flag:
            bcolors[index] = (204,226,203)
            if option&int(math.pow(2,1)):
                if root_report_num == '':
                    root_report_num = sentence
                    
                option_data.append(sentence)
                
            flag = False

        if sentence == '양식TQP-12-01-03(1)':
            bcolors[index-1] = (212,240,240)
            if option&int(math.pow(2,0)):
                option_data.append(sentences.pop())
            
        tmp = title_check(sentence)
        if tmp:
            bcolors[index] = (255,255,181)
            if option&int(math.pow(2,2)):
                option_data.append(sentence)
                

        if sentence == '성적서번호 : ':
            flag = True
            
        sentences.append(sentence)
            
    if option > 0:  
        result = option_data
        bcolors = [(255,255,255) for i in range(len(result))]

    else:
        result = sentences
        
    fcolors = [(0,0,0) for i in range(len(result))]
    if option == 0:
        for i, s in enumerate(result):
            pass
        
        
    if option == 1:
        chk_list = result[0].split()
        std_total_page = int(re.sub(r'[^0-9]', '', chk_list[1]))
        for i, s in enumerate(result):
            chk_list = s.split()
            if chk_list[0] != '총': fcolors[i] = (255,0,0)
            if '페이지' not in chk_list[1]: fcolors[i] = (255,0,0)
            if int(re.sub(r'[^0-9]', '', chk_list[1])) != std_total_page: fcolors[i] = (255,0,0)
            if chk_list[2] != '중': fcolors[i] = (255,0,0)
            if '페이지' not in chk_list[3]: fcolors[i] = (255,0,0)
            if int(re.sub(r'[^0-9]', '', chk_list[3])) != (i+2): fcolors[i] = (255,0,0)
                
    elif option == 2:
        std_number = result[0]
        for i, s in enumerate(result):
            if s != std_number: fcolors[i] = (255,0,0)
    
    return result, bcolors, fcolors

#r, b, f = Sentence_Extract('D:/Project/Jupyter/Document_Clustering/ImageHWP/Review_성적서을지_KCL_최종/성적서을지_KCL_최종.txt')

In [4]:
#------------------------------------------------ HWP 파일 변환 함수 ---------------------------------------------------

def hwp2etc(inout_path, output_path,option='TEXT'):
    if inout_path == None: return 0
    if output_path == None: return 0
    if option not in ('TEXT','PDF'): return 0
    
    hwp = win32.gencache.EnsureDispatch('HWPFrame.HwpObject')
    hwp.XHwpWindows.Item(0).Visible = False
    hwp.RegisterModule('FilePathCheckDLL', 'FilePathCheckerModule')
    hwp.Open(inout_path)
    hwp.FindCtrl()
    hwp.SaveAs(output_path,option)
    hwp.Clear(1)
    hwp.Quit()
    
    return 1

In [5]:
#------------------------------------------------ PDF 파일 이미지 변환 함수 ---------------------------------------------------

def pdf2img(image_path):
    
    images = convert_from_path(image_path, 
                               fmt='jpg', output_folder='D:/', 
                               dpi=600, 
                               #first_page=0, 
                               #last_page=9, 
                               poppler_path=r"Release-23.07.0-0\poppler-23.07.0\Library\bin")

    dir_path = os.path.dirname(image_path) + '/page_images'

    try:
        os.makedirs(dir_path)
    except OSError as exc: 
        if exc.errno == errno.EEXIST and os.path.isdir(dir_path):
            pass

    for idx, img in enumerate(images):
        img.save(dir_path + '/page' + str(idx) + '.jpg')
        
    return dir_path


In [6]:
#------------------------------------------------ GUI 코드 시작 ---------------------------------------------------

form_class = uic.loadUiType('KCL_Report_Reviewer.ui')[0]

#------------------------------------------------ UI 연결 및 함수 연결 ---------------------------------------------------
class MyWindow(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        
        self.setupUi(self)              
        self.setWindowTitle('KCL_성적서_검토도우미 v0.1.0')
        self.InputFile.clicked.connect(self.inputfile)
        self.OutputFile.clicked.connect(self.outputfile)
        self.TransformFile.clicked.connect(self.transformfile)
        self.TR_LoadFile.clicked.connect(self.loadfile_txt)
        self.IR_LoadFile.clicked.connect(self.loadfile_img)
        self.TR_ParseFile.clicked.connect(self.parsefile_txt)
        self.IR_ParseFile.clicked.connect(self.parsefile_img)
        self.Sentence_Up.clicked.connect(self.sentence_up)
        self.Sentence_Down.clicked.connect(self.sentence_down)
        self.Table_Sentence.cellClicked.connect(self.stable_cellclicked_event)
        self.Table_Word.cellClicked.connect(self.wtable_cellclicked_event)
        self.Zoom_Dial.valueChanged.connect(self.indicate_zoom)
        self.Zoom_Dial.setRange(-100,100)
        self.Page_Next.clicked.connect(self.page_next)
        self.Page_Prev.clicked.connect(self.page_prev)
        
#------------------------------------------------ 워크스페이스 생성 및 파일 변환 기능 ---------------------------------------------------

    def inputfile(self):
        fname = QFileDialog.getOpenFileName(self, '변환할 한글파일 선택', filter='*.hwp')
        self.InputLine.setText(fname[0])
        
    def outputfile(self):
        folder = QFileDialog.getExistingDirectory(self, '저장 디렉토리 선택')
        self.OutputLine.setText(folder)
        
    def transformfile(self):
        
        file_path = self.InputLine.text()
        file_name = os.path.basename(file_path)
        dir_name = self.OutputLine.text() + '/Review_' + file_name.replace('.hwp','')
        os.mkdir(dir_name)
        txt_path = dir_name + '/' + file_name.replace('.hwp','.txt')
        pdf_path = dir_name + '/' + file_name.replace('.hwp','.pdf')
        progress_string1 = 'Make Directory Review_' + file_name.replace('.hwp','') + ' is Success...'
        self.TransformLabel.setText(progress_string1)
        time.sleep(1)
        
        start = time.time()
        state = hwp2etc(file_path, txt_path)
        end = time.time()
        progress_string2 = 'Translate ' + file_name + ' to txt file, elapsed ' + str(round(end - start,5))+' s'
        self.TransformLabel.setText(progress_string1 + '\n' + progress_string2)
        time.sleep(1)
        
        start = time.time()
        state = hwp2etc(file_path, pdf_path, 'PDF')
        end = time.time()
        progress_string3 = 'Translate ' + file_name + ' to pdf file, elapsed ' + str(round(end - start,5))+' s'
        self.TransformLabel.setText(progress_string1 + '\n' + progress_string2 + '\n' + progress_string3)
#------------------------------------------------ 텍스트 검토 기능 ---------------------------------------------------

    def loadfile_txt(self):
        fname = QFileDialog.getOpenFileName(self, '검토할 텍스트파일 선택', filter='*.txt')
        self.TR_LoadLine.setText(fname[0])
        
    def parsefile_txt(self):
        
        option = (1 if self.Check_PageNum.isChecked() else 0) + (2 if self.Check_ReportNum.isChecked() else 0) + (4 if self.Check_Title.isChecked() else 0)
        lines, bcolors, fcolors = Sentence_Extract(self.TR_LoadLine.text(), option)
        self.Table_Sentence.setRowCount(len(lines))
        self.Table_Sentence.setColumnCount(1)
        self.Table_Sentence.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        for i in range(len(lines)):
            self.Table_Sentence.setItem(i,0,QTableWidgetItem(lines[i]))
            self.Table_Sentence.item(i,0).setBackground(QBrush(QColor(bcolors[i][0],bcolors[i][1],bcolors[i][2])))
            self.Table_Sentence.item(i,0).setForeground(QBrush(QColor(fcolors[i][0],fcolors[i][1],fcolors[i][2])))
            
    def display_wtable(self, row, col):
        data = self.Table_Sentence.item(row, col)
        self.Sentence_Display.setText(data.text())
        words = data.text().split()
        self.Table_Word.setRowCount(0)
        self.Table_Word.setColumnCount(4)
        self.Table_Word.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.Table_Word.setRowCount(int(len(words)/4)+1)
        for i in range(len(words)):
            self.Table_Word.setItem(int(i/4), i%4, QTableWidgetItem(words[i]))
            
    def stable_cellclicked_event(self,row,col):
        self.display_wtable(row, col)
            
    def wtable_cellclicked_event(self,row,col):
        word = self.Table_Word.item(row, col)
        if word is not None:
            QMessageBox.about(self,'About Title',word.text())
            
    def sentence_up(self):
        crow = self.Table_Sentence.currentRow()
        if crow <= 0:
            return
        self.Table_Sentence.selectRow(crow-1)
        self.display_wtable(crow-1, 0)
        
    def sentence_down(self):
        crow = self.Table_Sentence.currentRow()
        if crow >= (self.Table_Sentence.rowCount()-1):
            return
        self.Table_Sentence.selectRow(crow+1)
        self.display_wtable(crow+1, 0)
        
#------------------------------------------------ 이미지 검토 기능 ---------------------------------------------------
    def img_update(self, dir_path):
        pixmap = QPixmap(dir_path + '/'+self.Indicator_Page.text()+'.jpg')
        pixmap = pixmap.scaledToWidth(1200*(1+(self.Zoom_Dial.value()/100)))
        lbl_img = self.ImageView
        lbl_img.setPixmap(pixmap)
        lbl_img.setAlignment(Qt.AlignCenter)
        
    def loadfile_img(self):
        fname = QFileDialog.getOpenFileName(self, '검토할 PDF파일 선택', filter='*.pdf')
        self.IR_LoadLine.setText(fname[0])
        
    def parsefile_img(self):
        if self.Indicator_File.text() == '-':
            dir_path = pdf2img(self.IR_LoadLine.text())
            #dir_path = 'D:/Project/Jupyter/Document_Clustering/ImageHWP/Review_성적서을지_KCL_최종/page_images'
            self.Indicator_File.setText(dir_path)
        else:
            dir_path = self.Indicator_File.text()
        
        self.img_update(dir_path)
    
    def indicate_zoom(self):
        dir_path = self.Indicator_File.text()
        if dir_path == '-': return
        self.Indicator_Zoom.setText(str(self.Zoom_Dial.value())+' %')
        self.img_update(dir_path)
        
    def page_next(self):
        dir_path = self.Indicator_File.text()
        if dir_path == '-': return
        file_list = os.listdir(dir_path+'/')
        now_page = int(re.sub(r'[^0-9]', '', self.Indicator_Page.text()))
        chg_page = now_page + 1
        if 'page' + str(chg_page) + '.jpg' not in file_list: return
        self.Indicator_Page.setText('page' + str(chg_page))
        self.img_update(dir_path)
        
    def page_prev(self):
        dir_path = self.Indicator_File.text()
        if dir_path == '-': return
        file_list = os.listdir(dir_path)
        now_page = int(re.sub(r'[^0-9]', '', self.Indicator_Page.text()))
        chg_page = now_page - 1
        if 'page' + str(chg_page) + '.jpg' not in file_list: return
        self.Indicator_Page.setText('page' + str(chg_page))
        self.img_update(dir_path)


In [ ]:
if __name__ == '__main__':
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
        
    #font, ok = QFontDialog.getFont()
    app.setFont(QFont('경기천년제목 Light', 12))
    
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()




시험성적
시험성적
시험성적
성적서번호 :
성적서번호:
성적서번호:
CU23-01403
CU23-01403
CU-
고객이 의뢰한 적외선 뎁스 카메라 v1.0에 대하여 고객이 제시하는 시험항목, 시험절차, 시험환경에 따라 한국건설생활환경시험연구원이 수행하고 결과를 확인한다
고객이의뢰한적외선뎁스카메라v1.0에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
고객이의뢰한적외선뎁스카메라v.에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
      1
1

      2
2

      3
3

제품
제품
제품
의뢰
의뢰
의뢰
적외선 뎁스 카메라 v1.
적외선뎁스카메라v1.
적외선뎁스카메라v.
써모아이(주
써모아이(주
써모아이(주
구
구
구
시험항
시험항
시험항
개발목
개발목
개발목
단
단
단



적외선 뎁스 카메라 영상속
적외선뎁스카메라영상속
적외선뎁스카메라영상속
30 이
30이
이
fp
fp
fp



거리인식 정확
거리인식정확
거리인식정확
5 미
5미
미






야간 거리인식 정확
야간거리인식정확
야간거리인식정확
5 미
5미
미






역광 거리인식 정확
역광거리인식정확
역광거리인식정확
5 미
5미
미






터널 거리인식 정확
터널거리인식정확
터널거리인식정확
5 미
5미
미






비 거리인식 정확
비거리인식정확
비거리인식정확
5 미
5미
미






안개 거리인식 정확
안개거리인식정확
안개거리인식정확
5 미
5미
미






최대 거리인식범
최대거리인식범
최대거리인식범
250 이
250이
이






최소물체 인식크
최소물체인식크
최소물체인식크
35 × 25 이
35×25이
×이
c
c
c
1
1

영상정합
영상정합
영상정합
90 이
90이
이



1
1

인공지능 물체 인식
인공지능물체인식
인공지능물체인식
사
사
사
0.794 이
0.794이
.이



자전
자전
자전
0.580 이
0.580이
.이



차
차
차
0.856 이
0.856




시험성적
시험성적
시험성적
성적서번호 :
성적서번호:
성적서번호:
CU23-01403
CU23-01403
CU-
고객이 의뢰한 적외선 뎁스 카메라 v1.0에 대하여 고객이 제시하는 시험항목, 시험절차, 시험환경에 따라 한국건설생활환경시험연구원이 수행하고 결과를 확인한다
고객이의뢰한적외선뎁스카메라v1.0에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
고객이의뢰한적외선뎁스카메라v.에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
      1
1

      2
2

      3
3

제품
제품
제품
의뢰
의뢰
의뢰
적외선 뎁스 카메라 v1.
적외선뎁스카메라v1.
적외선뎁스카메라v.
써모아이(주
써모아이(주
써모아이(주
구
구
구
시험항
시험항
시험항
개발목
개발목
개발목
단
단
단



적외선 뎁스 카메라 영상속
적외선뎁스카메라영상속
적외선뎁스카메라영상속
30 이
30이
이
fp
fp
fp



거리인식 정확
거리인식정확
거리인식정확
5 미
5미
미






야간 거리인식 정확
야간거리인식정확
야간거리인식정확
5 미
5미
미






역광 거리인식 정확
역광거리인식정확
역광거리인식정확
5 미
5미
미






터널 거리인식 정확
터널거리인식정확
터널거리인식정확
5 미
5미
미






비 거리인식 정확
비거리인식정확
비거리인식정확
5 미
5미
미






안개 거리인식 정확
안개거리인식정확
안개거리인식정확
5 미
5미
미






최대 거리인식범
최대거리인식범
최대거리인식범
250 이
250이
이






최소물체 인식크
최소물체인식크
최소물체인식크
35 × 25 이
35×25이
×이
c
c
c
1
1

영상정합
영상정합
영상정합
90 이
90이
이



1
1

인공지능 물체 인식
인공지능물체인식
인공지능물체인식
사
사
사
0.794 이
0.794이
.이



자전
자전
자전
0.580 이
0.580이
.이



차
차
차
0.856 이
0.856




시험성적
시험성적
시험성적
성적서번호 :
성적서번호:
성적서번호:
CU23-01403
CU23-01403
CU-
고객이 의뢰한 적외선 뎁스 카메라 v1.0에 대하여 고객이 제시하는 시험항목, 시험절차, 시험환경에 따라 한국건설생활환경시험연구원이 수행하고 결과를 확인한다
고객이의뢰한적외선뎁스카메라v1.0에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
고객이의뢰한적외선뎁스카메라v.에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
      1
1

      2
2

      3
3

제품
제품
제품
의뢰
의뢰
의뢰
적외선 뎁스 카메라 v1.
적외선뎁스카메라v1.
적외선뎁스카메라v.
써모아이(주
써모아이(주
써모아이(주
구
구
구
시험항
시험항
시험항
개발목
개발목
개발목
단
단
단



적외선 뎁스 카메라 영상속
적외선뎁스카메라영상속
적외선뎁스카메라영상속
30 이
30이
이
fp
fp
fp



거리인식 정확
거리인식정확
거리인식정확
5 미
5미
미






야간 거리인식 정확
야간거리인식정확
야간거리인식정확
5 미
5미
미






역광 거리인식 정확
역광거리인식정확
역광거리인식정확
5 미
5미
미






터널 거리인식 정확
터널거리인식정확
터널거리인식정확
5 미
5미
미






비 거리인식 정확
비거리인식정확
비거리인식정확
5 미
5미
미






안개 거리인식 정확
안개거리인식정확
안개거리인식정확
5 미
5미
미






최대 거리인식범
최대거리인식범
최대거리인식범
250 이
250이
이






최소물체 인식크
최소물체인식크
최소물체인식크
35 × 25 이
35×25이
×이
c
c
c
1
1

영상정합
영상정합
영상정합
90 이
90이
이



1
1

인공지능 물체 인식
인공지능물체인식
인공지능물체인식
사
사
사
0.794 이
0.794이
.이



자전
자전
자전
0.580 이
0.580이
.이



차
차
차
0.856 이
0.856




시험성적
시험성적
시험성적
성적서번호 :
성적서번호:
성적서번호:
CU23-01403
CU23-01403
CU-
고객이 의뢰한 적외선 뎁스 카메라 v1.0에 대하여 고객이 제시하는 시험항목, 시험절차, 시험환경에 따라 한국건설생활환경시험연구원이 수행하고 결과를 확인한다
고객이의뢰한적외선뎁스카메라v1.0에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
고객이의뢰한적외선뎁스카메라v.에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
      1
1

      2
2

      3
3

제품
제품
제품
의뢰
의뢰
의뢰
적외선 뎁스 카메라 v1.
적외선뎁스카메라v1.
적외선뎁스카메라v.
써모아이(주
써모아이(주
써모아이(주
구
구
구
시험항
시험항
시험항
개발목
개발목
개발목
단
단
단



적외선 뎁스 카메라 영상속
적외선뎁스카메라영상속
적외선뎁스카메라영상속
30 이
30이
이
fp
fp
fp



거리인식 정확
거리인식정확
거리인식정확
5 미
5미
미






야간 거리인식 정확
야간거리인식정확
야간거리인식정확
5 미
5미
미






역광 거리인식 정확
역광거리인식정확
역광거리인식정확
5 미
5미
미






터널 거리인식 정확
터널거리인식정확
터널거리인식정확
5 미
5미
미






비 거리인식 정확
비거리인식정확
비거리인식정확
5 미
5미
미






안개 거리인식 정확
안개거리인식정확
안개거리인식정확
5 미
5미
미






최대 거리인식범
최대거리인식범
최대거리인식범
250 이
250이
이






최소물체 인식크
최소물체인식크
최소물체인식크
35 × 25 이
35×25이
×이
c
c
c
1
1

영상정합
영상정합
영상정합
90 이
90이
이



1
1

인공지능 물체 인식
인공지능물체인식
인공지능물체인식
사
사
사
0.794 이
0.794이
.이



자전
자전
자전
0.580 이
0.580이
.이



차
차
차
0.856 이
0.856




시험성적
시험성적
시험성적
성적서번호 :
성적서번호:
성적서번호:
CU23-01403
CU23-01403
CU-
고객이 의뢰한 적외선 뎁스 카메라 v1.0에 대하여 고객이 제시하는 시험항목, 시험절차, 시험환경에 따라 한국건설생활환경시험연구원이 수행하고 결과를 확인한다
고객이의뢰한적외선뎁스카메라v1.0에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
고객이의뢰한적외선뎁스카메라v.에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
      1
1

      2
2

      3
3

제품
제품
제품
의뢰
의뢰
의뢰
적외선 뎁스 카메라 v1.
적외선뎁스카메라v1.
적외선뎁스카메라v.
써모아이(주
써모아이(주
써모아이(주
구
구
구
시험항
시험항
시험항
개발목
개발목
개발목
단
단
단



적외선 뎁스 카메라 영상속
적외선뎁스카메라영상속
적외선뎁스카메라영상속
30 이
30이
이
fp
fp
fp



거리인식 정확
거리인식정확
거리인식정확
5 미
5미
미






야간 거리인식 정확
야간거리인식정확
야간거리인식정확
5 미
5미
미






역광 거리인식 정확
역광거리인식정확
역광거리인식정확
5 미
5미
미






터널 거리인식 정확
터널거리인식정확
터널거리인식정확
5 미
5미
미






비 거리인식 정확
비거리인식정확
비거리인식정확
5 미
5미
미






안개 거리인식 정확
안개거리인식정확
안개거리인식정확
5 미
5미
미






최대 거리인식범
최대거리인식범
최대거리인식범
250 이
250이
이






최소물체 인식크
최소물체인식크
최소물체인식크
35 × 25 이
35×25이
×이
c
c
c
1
1

영상정합
영상정합
영상정합
90 이
90이
이



1
1

인공지능 물체 인식
인공지능물체인식
인공지능물체인식
사
사
사
0.794 이
0.794이
.이



자전
자전
자전
0.580 이
0.580이
.이



차
차
차
0.856 이
0.856




시험성적
시험성적
시험성적
성적서번호 :
성적서번호:
성적서번호:
CU23-01403
CU23-01403
CU-
고객이 의뢰한 적외선 뎁스 카메라 v1.0에 대하여 고객이 제시하는 시험항목, 시험절차, 시험환경에 따라 한국건설생활환경시험연구원이 수행하고 결과를 확인한다
고객이의뢰한적외선뎁스카메라v1.0에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
고객이의뢰한적외선뎁스카메라v.에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
      1
1

      2
2

      3
3

제품
제품
제품
의뢰
의뢰
의뢰
적외선 뎁스 카메라 v1.
적외선뎁스카메라v1.
적외선뎁스카메라v.
써모아이(주
써모아이(주
써모아이(주
구
구
구
시험항
시험항
시험항
개발목
개발목
개발목
단
단
단



적외선 뎁스 카메라 영상속
적외선뎁스카메라영상속
적외선뎁스카메라영상속
30 이
30이
이
fp
fp
fp



거리인식 정확
거리인식정확
거리인식정확
5 미
5미
미






야간 거리인식 정확
야간거리인식정확
야간거리인식정확
5 미
5미
미






역광 거리인식 정확
역광거리인식정확
역광거리인식정확
5 미
5미
미






터널 거리인식 정확
터널거리인식정확
터널거리인식정확
5 미
5미
미






비 거리인식 정확
비거리인식정확
비거리인식정확
5 미
5미
미






안개 거리인식 정확
안개거리인식정확
안개거리인식정확
5 미
5미
미






최대 거리인식범
최대거리인식범
최대거리인식범
250 이
250이
이






최소물체 인식크
최소물체인식크
최소물체인식크
35 × 25 이
35×25이
×이
c
c
c
1
1

영상정합
영상정합
영상정합
90 이
90이
이



1
1

인공지능 물체 인식
인공지능물체인식
인공지능물체인식
사
사
사
0.794 이
0.794이
.이



자전
자전
자전
0.580 이
0.580이
.이



차
차
차
0.856 이
0.856




시험성적
시험성적
시험성적
성적서번호 :
성적서번호:
성적서번호:
CU23-01403
CU23-01403
CU-
고객이 의뢰한 적외선 뎁스 카메라 v1.0에 대하여 고객이 제시하는 시험항목, 시험절차, 시험환경에 따라 한국건설생활환경시험연구원이 수행하고 결과를 확인한다
고객이의뢰한적외선뎁스카메라v1.0에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
고객이의뢰한적외선뎁스카메라v.에대하여고객이제시하는시험항목,시험절차,시험환경에따라한국건설생활환경시험연구원이수행하고결과를확인한다
      1
1

      2
2

      3
3

제품
제품
제품
의뢰
의뢰
의뢰
적외선 뎁스 카메라 v1.
적외선뎁스카메라v1.
적외선뎁스카메라v.
써모아이(주
써모아이(주
써모아이(주
구
구
구
시험항
시험항
시험항
개발목
개발목
개발목
단
단
단



적외선 뎁스 카메라 영상속
적외선뎁스카메라영상속
적외선뎁스카메라영상속
30 이
30이
이
fp
fp
fp



거리인식 정확
거리인식정확
거리인식정확
5 미
5미
미






야간 거리인식 정확
야간거리인식정확
야간거리인식정확
5 미
5미
미






역광 거리인식 정확
역광거리인식정확
역광거리인식정확
5 미
5미
미






터널 거리인식 정확
터널거리인식정확
터널거리인식정확
5 미
5미
미






비 거리인식 정확
비거리인식정확
비거리인식정확
5 미
5미
미






안개 거리인식 정확
안개거리인식정확
안개거리인식정확
5 미
5미
미






최대 거리인식범
최대거리인식범
최대거리인식범
250 이
250이
이






최소물체 인식크
최소물체인식크
최소물체인식크
35 × 25 이
35×25이
×이
c
c
c
1
1

영상정합
영상정합
영상정합
90 이
90이
이



1
1

인공지능 물체 인식
인공지능물체인식
인공지능물체인식
사
사
사
0.794 이
0.794이
.이



자전
자전
자전
0.580 이
0.580이
.이



차
차
차
0.856 이
0.856